In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from models.sentence_transformer.bert_sentence_dataset import SentenceBertDataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df['name_1'] = df['name_1'].str.lower()
df['name_2'] = df['name_2'].str.lower()

In [ ]:
names = pd.concat([df['name_1'], df['name_2']])

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataset = SentenceBertDataset(df)

train_dataloader = DataLoader(
    train_dataset, 
    sampler=RandomSampler(train_dataset), 
    batch_size=32
)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
from sentence_transformers import SentenceTransformer, losses

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
train_loss = losses.CosineSimilarityLoss(model)
model.max_seq_length = 128

In [ ]:
num_epochs = 1
warmup_steps = np.ceil(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path='../weights/sbert_model'
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/14002 [00:00<?, ?it/s]

In [ ]:
names = names.drop_duplicates().values
embeddings = model.encode(names)

In [ ]:
import pickle
with open('/content/drive/MyDrive/embeddings.pkl', "wb") as fOut:
    pickle.dump({"names": names, "embeddings": embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/MyDrive/embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_names = stored_data['names']
    stored_embeddigs = stored_data['embeddings']

In [ ]:
sbert = SentenceTransformer('../weights/sbert_model', device='cuda:0')

In [ ]:
embedding = sbert.encode(df.loc[0, 'name_2'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine = cosine_similarity([embedding], stored_embeddigs)[0]

In [ ]:
similar_idx = np.argsort(cosine)[::-1][:5]
stored_names[similar_idx]